In [8]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import numpy as np
import kaleido

In [9]:
df = pd.read_csv('../results/clip_analysis.csv')

In [10]:
df['condition'] = np.where(df['word'].str.lower() == df['color'].str.lower(), 'congruent', 'incongruent')
df.loc[df['word'] == 'XXXX', 'condition'] = 'neutral'
df



,word,color,class_set,class_name,probability,condition
0,BLUE,red,word_only,the word is blue,0.985489,incongruent
1,BLUE,red,word_only,the word is red,0.014511,incongruent
2,BLUE,red,color_only,the color is blue,0.968680,incongruent
3,BLUE,red,color_only,the color is red,0.031320,incongruent
4,BLUE,red,combined,the word is blue in red color,0.766408,incongruent
5,BLUE,red,combined,the word is blue in blue color,0.129493,incongruent
6,BLUE,red,combined,the word is red in blue color,0.100890,incongruent
7,BLUE,red,combined,the word is red in red color,0.003209,incongruent
8,BLUE,blue,word_only,the word is blue,0.998753,congruent
9,BLUE,blue,word_only,the word is red,0.001247,congruent


In [11]:
class_names = df['class_name'].unique()
fig = px.bar(df, 
             y='probability', 
             x='class_name', 
             facet_col='class_set', 
             color='color', 
             color_discrete_map={'blue': 'blue', 'red': 'red'},
             facet_row='word',
             hover_data=df.columns.tolist(),
             barmode='group', category_orders={'class_name': class_names})
# the the x-axis does not need to be the same
fig.update_xaxes(matches=None) 
for col in range(1, 4):
    for row in range(2, 4):
        fig.update_xaxes(matches=f'x{col}', row=row, col=col)  # x1 matches x2
fig.update_layout(height=800, width=1200, title_text='CLIP Stroop Test')
fig.show()
import plotly.io as pio   
pio.kaleido.scope.mathjax = None
fig.write_image('clip_stroop_test.pdf')


In [ ]:
# anova
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm


In [60]:
import pingouin

In [72]:
# analyzing the wrod only and color only conditions:

def extended_anova(df, equation):
    model = ols(equation, data=df).fit()
    anova_table = anova_lm(model)
    # Calculate eta squared
    ss_total = anova_table['sum_sq'].sum()
    ss_error = anova_table.loc['Residual', 'sum_sq']
    anova_table['eta_sq'] = anova_table['sum_sq'] / ss_total
    anova_table['partial_eta_sq'] = anova_table['sum_sq'] / (anova_table['sum_sq'] + ss_error)
    anova_table['cohen_f'] = np.sqrt(anova_table['partial_eta_sq'] / (1 - anova_table['partial_eta_sq']))
    return anova_table
    



for class_set in ['word_only', 'color_only']:
    print(class_set)
    subset = df.query(f'(class_set == "{class_set}") & (word != "XXXX")').copy()
    subset.loc[:, 'response'] = np.where(subset['class_name'].str.contains('blue'), 'blue', 'red')
    anova_table = extended_anova(subset, 'probability ~  C(word) * C(response) + C(color)*C(response)')
    display(anova_table)



word_only


,df,sum_sq,mean_sq,F,PR(>F),eta_sq,partial_eta_sq,cohen_f
C(word),1.0,1.729828e-15,1.729828e-15,7.230957e-10,0.999981,8.883044e-16,3.615478e-10,0.000019
C(response),1.0,1.181030e-05,1.181030e-05,4.936892e+00,0.156385,6.064845e-06,7.116865e-01,1.571129
C(color),1.0,2.069306e-16,2.069306e-16,8.650027e-11,0.999993,1.062634e-16,4.325013e-11,0.000007
C(word):C(response),1.0,1.947046e+00,1.947046e+00,8.138962e+05,0.000001,9.998505e-01,9.999975e-01,637.924832
C(color):C(response),1.0,2.745798e-04,2.745798e-04,1.147787e+02,0.008600,1.410027e-04,9.828736e-01,7.575577
Residual,2.0,4.784507e-06,2.392253e-06,NaN,NaN,2.456948e-06,5.000000e-01,1.000000


color_only


,df,sum_sq,mean_sq,F,PR(>F),eta_sq,partial_eta_sq,cohen_f
C(word),1.0,4.282768e-15,4.282768e-15,8.317015e-11,0.999994,2.249893e-15,4.158507e-11,0.000006
C(response),1.0,1.614999e-04,1.614999e-04,3.136284e+00,0.218582,8.484178e-05,6.106134e-01,1.252255
C(color),1.0,5.504020e-16,5.504020e-16,1.068865e-11,0.999998,2.891462e-16,5.344326e-12,0.000002
C(word):C(response),1.0,1.902332e+00,1.902332e+00,3.694274e+04,0.000027,9.993638e-01,9.999459e-01,135.909425
C(color):C(response),1.0,9.465404e-04,9.465404e-04,1.838155e+01,0.050331,4.972520e-04,9.018720e-01,3.031629
Residual,2.0,1.029881e-04,5.149405e-05,NaN,NaN,5.410339e-05,5.000000e-01,1.000000


In [77]:
# statistical analysis

subset = df.query('(class_set in ["word_only", "color_only"]) & (word == "XXXX")').copy()
subset.loc[:, 'response'] = np.where(subset['class_name'].str.contains('blue'), 'blue', 'red')
anova_table = extended_anova(subset, 'probability ~  C(color)*C(response)')
display(anova_table)


,df,sum_sq,mean_sq,F,PR(>F),eta_sq,partial_eta_sq,cohen_f
C(color),1.0,1.667069e-15,1.667069e-15,4.467583e-11,9.999950e-01,1.150703e-15,1.116896e-11,0.000003
C(response),1.0,5.146677e-03,5.146677e-03,1.379259e+02,3.007143e-04,3.552518e-03,9.718163e-01,5.872093
C(color):C(response),1.0,1.443444e+00,1.443444e+00,3.868290e+04,4.009020e-09,9.963445e-01,9.998966e-01,98.339850
Residual,4.0,1.492592e-04,3.731479e-05,NaN,NaN,1.030269e-04,5.000000e-01,1.000000


In [78]:
subset = df.query('(class_set in ["combined"]) & (word != "XXXX")').copy()
subset.loc[:, 'response_word'] = subset['class_name'].str.extract('word is (\\w+)')
subset.loc[:, 'response_color'] = subset['class_name'].str.extract('(\\w+) color')
anova_table = extended_anova(subset, 'probability ~  C(word) * C(response_word) + C(color)*C(response_color) + C(word)*C(response_color) + C(color)*C(response_word)')
display(anova_table)


,df,sum_sq,mean_sq,F,PR(>F),eta_sq,partial_eta_sq,cohen_f
C(word),1.0,2.594039e-17,2.594039e-17,6.523285e-16,1.000000,2.455348e-17,9.318978e-17,9.653485e-09
C(response_word),1.0,4.541195e-03,4.541195e-03,1.141984e-01,0.745322,4.298400e-03,1.605218e-02,1.277265e-01
C(color),1.0,2.723118e-16,2.723118e-16,6.847883e-15,1.000000,2.577526e-16,9.782690e-16,3.127729e-08
C(response_color),1.0,1.098783e-02,1.098783e-02,2.763133e-01,0.615363,1.040037e-02,3.797436e-02,1.986790e-01
C(word):C(response_word),1.0,5.885323e-01,5.885323e-01,1.479995e+01,0.006316,5.570664e-01,6.788983e-01,1.454056e+00
C(color):C(response_color),1.0,1.691197e-01,1.691197e-01,4.252889e+00,0.078105,1.600777e-01,3.779375e-01,7.794585e-01
C(word):C(response_color),1.0,5.620330e-05,5.620330e-05,1.413356e-03,0.971061,5.319839e-05,2.018673e-04,1.420944e-02
C(color):C(response_word),1.0,4.886975e-03,4.886975e-03,1.228938e-01,0.736229,4.625693e-03,1.725335e-02,1.325000e-01
Residual,7.0,2.783608e-01,3.976583e-02,NaN,NaN,2.634783e-01,5.000000e-01,1.000000e+00


In [79]:
# repeat for XXXX
subset = df.query('(class_set in ["combined"]) & (word == "XXXX")').copy()
subset.loc[:, 'response_word'] = subset['class_name'].str.extract('word is (\\w+)')
subset.loc[:, 'response_color'] = subset['class_name'].str.extract('(\\w+) color')
anova_table = extended_anova(subset, 'probability ~  C(color)*C(response_color) + C(color)*C(response_word)')
display(anova_table)

,df,sum_sq,mean_sq,F,PR(>F),eta_sq,partial_eta_sq,cohen_f
C(color),1.0,5.013351e-16,5.013351e-16,3.526719e-14,1.000000,1.581440e-15,1.763359e-14,1.327915e-07
C(response_color),1.0,4.728045e-05,4.728045e-05,3.326016e-03,0.959254,1.491442e-04,1.660247e-03,4.077999e-02
C(response_word),1.0,2.591455e-02,2.591455e-02,1.822999e+00,0.309456,8.174636e-02,4.768505e-01,9.547249e-01
C(color):C(response_color),1.0,1.506669e-01,1.506669e-01,1.059890e+01,0.082800,4.752724e-01,8.412559e-01,2.302053e+00
C(color):C(response_word),1.0,1.119523e-01,1.119523e-01,7.875455e+00,0.106984,3.531487e-01,7.974777e-01,1.984371e+00
Residual,2.0,2.843068e-02,1.421534e-02,NaN,NaN,8.968338e-02,5.000000e-01,1.000000e+00
